In [1]:
using Distributed
# @everywhere used for the implementation of the modules across multiple workers
# Without it, the code does not run

# Add the modules to the ´read´ path of the Julia
@everywhere push!(LOAD_PATH, "/scratch/users/vaydingul20/workfolder/COMP541_Project/src/modules/")


# Custom modules
@everywhere using Model: HapticNet, VisualNet, train_epoch!, save_as_jld2
@everywhere using TUM69: load_accel_data, load_image_data
@everywhere using Preprocess: process_accel_signal, process_image, augment_image
@everywhere using Utils: kfold, notify!, a_type
# Ready-to-use modules
using JLD2, Random
using CUDA
using Augmentor: FlipX, FlipY

┌ Info: Precompiling Preprocess [top-level]
└ @ Base loading.jl:1278


In [2]:
# Set path
path = CUDA.functional() ? "/userfiles/vaydingul20/data/new" : "data/new" # path of the main data
DATA_PATH = isdir(path) && path
# Set pretrained network path
alexnet_PATH = "/scratch/users/vaydingul20/workfolder/COMP541_Project/alexnet.mat"

"/scratch/users/vaydingul20/workfolder/COMP541_Project/alexnet.mat"

In [5]:
# Load image data
X_train, y_train, _, _, material_dict = @time load_image_data(DATA_PATH; mode="normal");

Train image data is being loaded!
Test image data is being loaded!
495.914231 seconds (286.47 k allocations: 92.438 GiB, 3.00% gc time)


In [6]:
# Augmentation pipeline
p1 = FlipX()
p2 = FlipY()
p3 = FlipX() |> FlipY()
X_train, y_train = @time augment_image(X_train, y_train, p1, p2, p3);

393.140817 seconds (1.26 M allocations: 46.275 GiB, 1.46% gc time)


In [ ]:
# Apply preprocessing on the images
# Here is where the code explodes
X_train, y_train = @time process_image(X_train, y_train);

In [ ]:
# Seperate into 3 folds for training
kf = kfold(X_train, y_train; fold=3, minibatch_size = 10, atype=a_type(Float32))
results = []

In [21]:
for (ix, (dtrn, dtst)) in enumerate(kf.folds[1:1])
    

    #Reset the model
    vn = VisualNet(alexnet_PATH; atype=a_type(Float32))

    for k in 1:1
        # Training routine
        res = train_epoch!(vn, dtrn, dtst; progress_bar=true, fig=false, info=true, epoch=100)
        # Save model 
        # Add results to the ´results´vector
        push!(results, res)

    end
    save_as_jld2(vn, "vn-$ix.jld2")

end

┣████████████████████┫ [100.00%, 4800/4800, 03:37/03:37, 22.10i/s] 


TRAINING PARAMETERS


Activation Function = relu
Optimizer Type = adam
Learning Rate = 0.0001
LAYERS:


Model.Conv ==> W = (11, 11, 3, 48)   b = (1, 1, 48, 1)
Model.Conv ==> W = (5, 5, 48, 256)   b = (1, 1, 256, 1)
Model.Conv ==> W = (3, 3, 256, 192)   b = (1, 1, 192, 1)
Model.Conv ==> W = (3, 3, 192, 192)   b = (1, 1, 192, 1)
Model.Conv ==> W = (3, 3, 192, 256)   b = (1, 1, 256, 1)
Model.Conv ==> W = (6, 6, 256, 300)   b = (1, 1, 300, 1)
Model.Conv ==> W = (1, 1, 300, 250)   b = (1, 1, 250, 1)
Model.Conv ==> W = (1, 1, 250, 69)   b = (1, 1, 69, 1)
In this network configuration,
there are total 4399749 parameters.
Final Loss
Train Loss = 0.00023503655
Test Loss = 0.0010540287


Final Misclassification Error
Train Misclassification Error = 0.0
Test Misclassification Error = 0.0


Test Accuracy = 1.0


In [11]:
# Save accumulated results
JLD2.@save "results_vn.jld2" results = results